<a href="https://colab.research.google.com/github/gazabAnurag/TF_BETARUNS/blob/master/LSTM_TEXT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unicode

In [0]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import os
import re
import random
import unidecode
import time

In [3]:
name,path='shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
file = tf.keras.utils.get_file(name,path)

1122304/1115394 [==============================] - 0s 0us/step


In [4]:
text = unidecode.unidecode(open(file).read())
len(text)

1115394

In [0]:
textsorted = sorted(set(text))
idx2chr = {k:v for (k,v) in enumerate(textsorted)}
chr2idx = {v:k for (k,v) in enumerate(textsorted)}

In [0]:
max_length = 150
vocab_size = len(textsorted)
embedding_dim = 256
gru_units=1024
BATCH_SIZE=64
BUFFER_SIZE= 10000

In [0]:
#create batches for input and output tensor
input_text = []
output_text = []
for i in range(0,len(text)-max_length,max_length):
  temp_in= text[i:i+max_length]
  temp_out= text[i+1:i+1+max_length]
  input_text.append([chr2idx[i] for i in temp_in])
  output_text.append([chr2idx[i] for i in temp_out])

In [8]:
len(np.array(output_text).shape)

2

In [0]:
data = tf.data.Dataset.from_tensor_slices((input_text,output_text)).shuffle(BUFFER_SIZE)
data = data.batch(BATCH_SIZE, drop_remainder=True)

In [10]:
print(data)

<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int32, tf.int32)>


In [0]:
class Model(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,units,batch_size):
    super(Model,self).__init__()
    self.units=units
    self.batch_sz=batch_size
    self.embedding =tf.keras.layers.Embedding(vocab_size,embedding_dim)
    #check if the GPU is availaible using tf.test.is_gpu_available()
    if tf.test.is_gpu_available():
      self.gru = tf.keras.layers.CuDNNGRU(self.units, 
                                          return_sequences=True, 
                                          return_state=True, 
                                          recurrent_initializer='glorot_uniform')
    else:
      self.gru = tf.keras.layers.GRU(self.units, 
                                     return_sequences=True, 
                                     return_state=True, 
                                     recurrent_activation='sigmoid', 
                                     recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
  def call(self,x,hidden):
    x = self.embedding(x)
    output,state = self.gru(x,initial_state = hidden)
    output = tf.reshape(output,(-1,output.shape[2]))
    x= self.fc(output)
    return x,state
  

In [0]:
model = Model(vocab_size,embedding_dim,gru_units,BATCH_SIZE)

In [0]:
optimizer = tf.train.AdamOptimizer()
def loss_function(real,pred):
  return tf.losses.sparse_softmax_cross_entropy(labels=real,logits=pred)

checkpoint_dir = './train_checkpoint'
checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,model=model)

In [14]:
EPOCHS = 20

for epoch in range(EPOCHS):
    start = time.time()
    
    # initializing the hidden state at the start of every epoch
    hidden = model.reset_states()
    
    for (batch, (inp, target)) in enumerate(data):
          with tf.GradientTape() as tape:
              # feeding the hidden state back into the model
              # This is the interesting step
              predictions, hidden = model(inp, hidden)
              
              # reshaping the target because that's how the 
              # loss function expects it
              target = tf.reshape(target, (-1,))
              loss = loss_function(target, predictions)
              
          grads = tape.gradient(loss, model.variables)
          optimizer.apply_gradients(zip(grads, model.variables))

          if batch % 100 == 0:
              print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch+1,
                                                            batch,
                                                            loss))
    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.1750
Epoch 1 Batch 100 Loss 2.3121
Epoch 1 Loss 2.2693
Time taken for 1 epoch 25.119115591049194 sec

Epoch 2 Batch 0 Loss 2.2700
Epoch 2 Batch 100 Loss 2.0042
Epoch 2 Loss 1.9599
Time taken for 1 epoch 22.764246940612793 sec

Epoch 3 Batch 0 Loss 1.9052
Epoch 3 Batch 100 Loss 1.7613
Epoch 3 Loss 1.7419
Time taken for 1 epoch 22.606393814086914 sec

Epoch 4 Batch 0 Loss 1.7334
Epoch 4 Batch 100 Loss 1.6387
Epoch 4 Loss 1.6016
Time taken for 1 epoch 23.487173557281494 sec

Epoch 5 Batch 0 Loss 1.5910
Epoch 5 Batch 100 Loss 1.5231
Epoch 5 Loss 1.5267
Time taken for 1 epoch 22.91302800178528 sec

Epoch 6 Batch 0 Loss 1.4725
Epoch 6 Batch 100 Loss 1.4577
Epoch 6 Loss 1.4419
Time taken for 1 epoch 22.95051860809326 sec

Epoch 7 Batch 0 Loss 1.3990
Epoch 7 Batch 100 Loss 1.4161
Epoch 7 Loss 1.3781
Time taken for 1 epoch 22.668579578399658 sec

Epoch 8 Batch 0 Loss 1.3425
Epoch 8 Batch 100 Loss 1.3671
Epoch 8 Loss 1.3624
Time taken for 1 epoch 22.68913722038269 sec

Epo

In [15]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [21]:
# Evaluation step(generating text using the model learned)

# number of characters to generate
num_generate = 1000

# You can change the start string to experiment
start_string = 'Q'
# converting our start string to numbers(vectorizing!) 
input_eval = [chr2idx[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)

# empty string to store our results
text_generated = ''

# hidden state shape == (batch_size, number of rnn units); here batch size == 1
hidden = [tf.zeros((1, gru_units))]
for i in range(num_generate):
    predictions, hidden = model(input_eval, hidden)

    # using argmax to predict the word returned by the model
    predicted_id = tf.argmax(predictions[-1]).numpy()
    
    # We pass the predicted word as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)
    
    text_generated += idx2chr[predicted_id]

print (start_string + text_generated)

QUEY:
I have done this world to hear thee hence,
And there the court of the morning of a fool.

GLOUCESTER:
Why, then he shall not stay to thee and the princess of the courtier.

Shepherd:
What, will you go with me?

DUKE VINCENTIO:
I know you well, so hath the fair days of the world,
The manner of your son, the thing I have the more
Than when the sense of honour and the court
And so he came from him as he hath done thee how to say
When he shall be the fair daughter of a king,
And then he says he is coming.

BRUTUS:
We have a man to do the gods;
And then have ta'en the streets as free from hence,
And there the court of the morning of a fool.

GLOUCESTER:
Why, then he shall not stay to thee and the princess of the courtier.

Shepherd:
What, will you go with me?

DUKE VINCENTIO:
I know you well, so hath the fair days of the world,
The manner of your son, the thing I have the more
Than when the sense of honour and the court
And so he came from him as he hath done thee how to say
When he s